In [ ]:
# Copyright 2025 DeepMind Technologies Limited. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google/genai-processors/blob/main/notebooks/constrained_decoding_intro.ipynb)

# Structured Clarity: Dataclasses as a Modality

Large language models are great at generating text, but what if you need to work with structured data like Python `dataclasses`? The GenAI Processors library treats your custom data types as just another modality, like text or images.

This makes it trivial to create pipelines that consume, produce, and transform structured data. The key features are:

*   **Dataclasses as a Modality:** Your custom `dataclasses` can be easily packed into and unpacked from `ProcessorPart` objects using `ProcessorPart.from_dataclass()` and `part.get_dataclass()`. The underlying representation is simply JSON.
*   **Automatic Model Integration:** When using `GenaiModel` or `OllamaModel`, you can simply specify a `response_schema` (e.g., `response_schema=MyDataclass`). The library automatically handles the constrained decoding request and parses the model's JSON output into typed `ProcessorPart` objects, ready for you to use. If the schema is a list, each item is yielded as a separate part, enabling concurrent processing.

This notebook will walk you through these features, showing how to seamlessly integrate structured data into your AI workflows.

## 1. ⚙️ Setup

First, let's install the GenAI Processors library and its dependencies.

In [ ]:
!pip install -q genai-processors

### API Key

To use the GenAI model processors, you'll need a Gemini API key. If you haven't already, get your key from Google AI Studio and add it as a secret in Colab (recommended) or set it directly below.

In [ ]:
import os
from google.colab import userdata

try:
  os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
except userdata.SecretNotFoundError:
  print(
      'GOOGLE_API_KEY not found in Colab secrets. You can still run the'
      ' notebook, but the sections using GenaiModel will fail.'
  )

In [ ]:
import asyncio
import dataclasses
import enum

import dataclasses_json
from genai_processors import content_api
from genai_processors import processor
from genai_processors import streams
from genai_processors.core import constrained_decoding
from genai_processors.core import genai_model
from google.genai import types as genai_types
from IPython.display import Markdown, display
import nest_asyncio

nest_asyncio.apply()  # Needed to run async loops in Colab

ProcessorPart = content_api.ProcessorPart

## 2. 🎬 Defining Our Data Structures

First, let's define the Python data structures (`dataclasses` and `enums`) that we want our model to generate. These act as the "schema" for our desired output.

**Note:** For `dataclasses`, you must use the `@dataclasses_json.dataclass_json` decorator to enable automatic JSON serialization and deserialization.

In [ ]:
class Genre(enum.StrEnum):
  """Enum for movie genres."""

  SCI_FI = "Science Fiction"
  FANTASY = "Fantasy"
  ACTION = "Action"
  COMEDY = "Comedy"
  DRAMA = "Drama"


@dataclasses_json.dataclass_json
@dataclasses.dataclass(frozen=True)
class Actor:
  """Represents a single actor."""

  name: str
  birth_year: int


@dataclasses_json.dataclass_json
@dataclasses.dataclass(frozen=True)
class Movie:
  """Represents a movie with its details."""

  title: str
  release_year: int
  genre: Genre
  lead_actors: list[Actor]

## 3. 🧩 Custom Dataclasses as a Modality

The library is designed to let you treat your own data types as a first-class modality, just like text or images. You can easily pack a dataclass instance into a `ProcessorPart` and unpack it later.

This is useful for passing structured data between processors in a pipeline. Under the hood, the `ProcessorPart` simply stores the object as a JSON string, which means it's still compatible with any model that expects text.

In [ ]:
# 1. Create an instance of our dataclass.
movie_instance = Movie(
    title="The Matrix",
    release_year=1999,
    genre=Genre.SCI_FI,
    lead_actors=[Actor(name="Keanu Reeves", birth_year=1964)],
)

# 2. Pack it into a ProcessorPart.
part = ProcessorPart.from_dataclass(dataclass=movie_instance)

print("The underlying representation is just JSON:")
print(part.text)
print("\n---\n")

# 3. Unpack it back into a Python object.
unpacked_movie = part.get_dataclass(Movie)

print(f"Unpacked a '{type(unpacked_movie).__name__}' object:")
print(unpacked_movie)

# They are identical.
assert movie_instance == unpacked_movie

## 4. 🪄 Automatic Structured Output from Models

While packing and unpacking data is useful, the real power comes from getting structured objects directly from a language model.

With `GenaiModel` (and `OllamaModel`), you don't need to do any manual parsing. Simply provide your `dataclass` or `enum` as the `response_schema` in the model's configuration. The library will then automatically:

1.  Instruct the model to generate a JSON response matching your schema.
2.  Parse the incoming JSON stream.
3.  Yield `ProcessorPart` objects that are already packed with your dataclass instances.

### Example: Generating a Single Object

Let's ask the model to invent a movie and return it directly as a structured `Movie` object.

In [ ]:
# Configure the model to use our Movie dataclass as the response schema.
structured_movie_model = genai_model.GenaiModel(
    api_key=os.getenv("GOOGLE_API_KEY"),
    model_name="gemini-1.5-flash",
    generate_content_config=genai_types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=Movie,
        temperature=1.0,
    ),
)

prompt = (
    "Invent a plausible but fictional sci-fi movie. It should be a completely"
    " new concept."
)
output_parts = processor.apply_sync(structured_movie_model, [prompt])

movie_instance = output_parts[0].get_dataclass(Movie)
print(
    f"The model generated a '{type(movie_instance).__name__}' object"
    " directly!\n"
)
print(movie_instance)

### Example: Generating a List of Objects

This feature is even more powerful when working with lists. If you specify the target schema as a `list` (e.g., `list[Movie]`), the model processor will parse the JSON array and **yield each item as a separate `ProcessorPart`**.

This is incredibly useful for creating pipelines where subsequent processors can operate on each item individually and concurrently.

In [ ]:
# This time, the schema is a list of movies.
movie_list_model = genai_model.GenaiModel(
    api_key=os.getenv("GOOGLE_API_KEY"),
    model_name="gemini-1.5-flash",
    generate_content_config=genai_types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=list[Movie],
    ),
)

prompt = "Recommend two classic fantasy movies from the 1980s."
output_parts = processor.apply_sync(movie_list_model, [prompt])

print(
    f"The model returned {len(output_parts)} separate parts, one for each"
    " movie:"
)
for i, part in enumerate(output_parts):
  movie = part.get_dataclass(Movie)
  print(f"  Part {i+1}: {movie.title}")

## 5. 🧑‍🔬 Building a Pipeline with Structured Data

Now that we can reliably get structured `Movie` objects from the model, let's use them in a pipeline. Our agent will:

1.  Take a user prompt asking for movie recommendations.
2.  Use a `GenaiModel` configured to return a `list[Movie]`, which yields each `Movie` as a separate part.
3.  Chain a custom `PartProcessor` that takes each `Movie` object and formats it into a nice Markdown summary.
4.  Display the formatted summaries.

In [ ]:
@processor.part_processor_function
async def format_movie_summary(
    part: content_api.ProcessorPart,
) -> str:
  """Takes a ProcessorPart containing a Movie and yields a Markdown string."""
  movie = part.get_dataclass(Movie)  # Unpack the movie object from the part.
  if not movie:
    return  # Ignore any parts that aren't Movies.

  actor_list = ", ".join([actor.name for actor in movie.lead_actors])
  summary = (
      f"### {movie.title} ({movie.release_year})\n"
      f"**Genre**: {movie.genre.value}\n"
      f"**Starring**: {actor_list}\n"
      "---"
  )
  yield summary


movie_recommender_model = genai_model.GenaiModel(
    api_key=os.getenv("GOOGLE_API_KEY"),
    model_name="gemini-1.5-flash",
    generate_content_config=genai_types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=list[Movie],
    ),
)

recommendation_agent = movie_recommender_model + format_movie_summary

prompt = "Recommend three classic fantasy movies from the 1980s."

display(Markdown(f"**User Prompt:** *{prompt}*\n"))
display(Markdown("## Recommendations"))
async for result_part in recommendation_agent(
    processor.stream_content([prompt])
):
  display(Markdown(result_part.text))

## 6. ⚙️ Advanced: Under the Hood

For most use cases, the automatic handling of structured output shown above is all you need. However, for advanced scenarios, you might want to control the process more directly.

### Disabling Automatic Parsing

What if you provide a `response_schema` to guide the model's output, but you still want the raw JSON string instead of the parsed dataclass? You can achieve this by setting `stream_json=True` in the model processor's constructor. This disables the automatic parsing behavior.

In [ ]:
# This model will still request JSON from the API, but won't parse it.
raw_json_model = genai_model.GenaiModel(
    api_key=os.getenv("GOOGLE_API_KEY"),
    model_name="gemini-1.5-flash",
    generate_content_config=genai_types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=Movie,  # The API is still guided by this.
    ),
    stream_json=True,  # This is the key to disable parsing.
)

prompt = "Invent a plausible but fictional fantasy movie."
output_parts = processor.apply_sync(raw_json_model, [prompt])

raw_json_string = content_api.as_text(output_parts)
print("Got the raw JSON string from the model:\n")
print(raw_json_string)

### Direct Usage of the Parser

The automatic behavior is powered by a processor called `constrained_decoding.StructuredOutputParser`. While you typically don't need to use it directly, it can be useful if you have a stream of raw JSON from a source other than a model processor and want to parse it into typed `ProcessorPart` objects.

In [ ]:
# The StructuredOutputParser needs to know the target type.
json_parser = constrained_decoding.StructuredOutputParser(Movie)

json_input_stream = [
    ProcessorPart(
        '{"title": "The Matrix", "release_year": 1999, "genre": "Science'
        ' Fiction", '
    ),
    ProcessorPart(
        '"lead_actors": [{"name": "Keanu Reeves", "birth_year": 1964}]}'
    ),
]

output_parts = processor.apply_sync(json_parser, json_input_stream)
movie_instance = output_parts[0].get_dataclass(Movie)

print(f"Successfully parsed a '{type(movie_instance).__name__}' instance:")
print(movie_instance)

## 7. 🚀 Next Steps

You've now seen how to seamlessly work with structured data in GenAI Processors. This is a key technique for building reliable and predictable AI applications.

To continue your journey, explore these other notebooks:

*   [**Processor Introduction**](https://colab.research.google.com/github/google/genai-processors/blob/main/notebooks/processor_intro.ipynb): Get a foundational understanding of the core concepts of the library.
*   [**Create Your Own Processor**](https://colab.research.google.com/github/google/genai-processors/blob/main/notebooks/create_your_own_processor.ipynb): Learn how to build custom processors to create complex, multi-step AI pipelines.